<a href="https://colab.research.google.com/github/lauBit/proyecto-dos/blob/main/Evaluaci%C3%B3n_de_Calidad_de_Carros_Random_Forest_Versi%C3%B3n_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto 2 – Modelos de clasificación supervisada para evaluar la calidad de un automovil

Integrantes del Grupo:
- Paola Romero
- Fernando Montaño
- Laura Camelo
- Daniel Garcia

## **Explicación breve del problema**.
Se desea realizar un modelo de nivel de evaluación de vehiculos basado en caracteristicas tales como: precio de compra, precio de mantenimiento, número de puertas, cupo de personas, tamaño del maletero, seguridad estimada del vehiculo.

## Importación de Librerias

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import datasets, metrics

!pip install ucimlrepo

## Importación del Dataset "Car Evaluation"
El dataset se importa a través del paquete "ucimlrepo" y la librería "fetch_ucirepo" el cual se conecta directamente a la página web donde se encuentra alojado el dataset.

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
car_evaluation = fetch_ucirepo(id=19)

# data (as pandas dataframes)
X = car_evaluation.data.features
y = car_evaluation.data.targets

# metadata
print(car_evaluation.metadata)

# variable information
print(car_evaluation.variables)


{'uci_id': 19, 'name': 'Car Evaluation', 'repository_url': 'https://archive.ics.uci.edu/dataset/19/car+evaluation', 'data_url': 'https://archive.ics.uci.edu/static/public/19/data.csv', 'abstract': 'Derived from simple hierarchical decision model, this database may be useful for testing constructive induction and structure discovery methods.', 'area': 'Other', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 1728, 'num_features': 6, 'feature_types': ['Categorical'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1988, 'last_updated': 'Thu Aug 10 2023', 'dataset_doi': '10.24432/C5JP48', 'creators': ['Marko Bohanec'], 'intro_paper': {'title': 'Knowledge acquisition and explanation for multi-attribute decision making', 'authors': 'M. Bohanec, V. Rajkovič', 'published_in': '8th Intl Workshop on Expert Systems and their Applications, Avignon, France', 'yea

## Identificación de Variables y Registros
Se identifican 6 variables caracteristicas las cuales cuentan con un total de 1728 registros cada una. A continuación se relaciona el detalle de las variables mencionadas:
- `buying`: Precio de compra.
- `maint`: Precio del mantenimiento.
- `doors`: Número de puertas.
- `persons`: Cupo de personas.
- `lug_boot`: Tamaño del maletero.
- `safety`: Seguridad estimada del vehiculo.

Adicionalmente, se identifica que una variable objetivo llamada `class`, la cual cuenta con un total de 1728 registros al igual que lo identificado en las variables anteriores.



In [ ]:
print(X.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   object
 1   maint     1728 non-null   object
 2   doors     1728 non-null   object
 3   persons   1728 non-null   object
 4   lug_boot  1728 non-null   object
 5   safety    1728 non-null   object
dtypes: object(6)
memory usage: 81.1+ KB
None


In [ ]:
print(X.describe())

       buying  maint doors persons lug_boot safety
count    1728   1728  1728    1728     1728   1728
unique      4      4     4       3        3      3
top     vhigh  vhigh     2       2    small    low
freq      432    432   432     576      576    576


In [ ]:
print(y.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   class   1728 non-null   object
dtypes: object(1)
memory usage: 13.6+ KB
None


In [ ]:
print(y.describe())

        class
count    1728
unique      4
top     unacc
freq     1210


In [ ]:
print(y)

      class
0     unacc
1     unacc
2     unacc
3     unacc
4     unacc
...     ...
1723   good
1724  vgood
1725  unacc
1726   good
1727  vgood

[1728 rows x 1 columns]


Una vez validada la información de las variables del dataset, no se identifican datos atipicos asi como valores nulos, por lo cual no se realizara ajustes sobre la data.

## Estandarización de las Variables a Numéricas

In [ ]:
X['buyingAjustado'] = X['buying'].map({'vhigh': 3, 'high':2, 'med':1, 'low':0}).astype(int)
X['maintAjustado'] = X['maint'].map({'vhigh': 3, 'high':2, 'med':1, 'low':0}).astype(int)
X['doorsAjustado'] = X['doors'].map({'5more': 3, '4':2, '3':1, '2':0}).astype(int)
X['personsAjustado'] = X['persons'].map({'more': 2, '4':1, '2':0}).astype(int)
X['lug_bootAjustado'] = X['lug_boot'].map({'big': 2, 'med':1, 'small':0}).astype(int)
X['safetyAjustado'] = X['safety'].map({'low': 0, 'med': 1, 'high': 2}).astype(int)

## Implementación del Modelo "Random Forest"

### Partición de la data en conjuntos de entrenamiento y pruebas

In [ ]:
X_ajustado = X.drop(['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'lug_bootAjustado', 'doorsAjustado'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_ajustado, y, test_size=0.2, stratify=y, random_state=1)

### Entrenamiento del Modelo

In [ ]:
rf = RandomForestClassifier(n_estimators=100, min_samples_leaf=0.12, max_features=3, verbose=2, bootstrap=True, random_state=1)
rf.fit(X_train, y_train)

<ipython-input-23-125f80bd74df>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.1s


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

RandomForestClassifier(max_features=3, min_samples_leaf=0.12, random_state=1,
                       verbose=2)

### Predicción del Modelo

In [ ]:
y_pred = rf.predict(X_test)

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s


In [ ]:
accuracy_score(y_test, y_pred)

0.8005780346820809

In [ ]:
print(confusion_matrix(y_test, y_pred))

[[ 40   0  37   0]
 [ 14   0   0   0]
 [  5   0 237   0]
 [ 13   0   0   0]]


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         acc       0.56      0.52      0.54        77
        good       0.00      0.00      0.00        14
       unacc       0.86      0.98      0.92       242
       vgood       0.00      0.00      0.00        13

    accuracy                           0.80       346
   macro avg       0.36      0.37      0.36       346
weighted avg       0.73      0.80      0.76       346



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Conclusión Final - Validación Modelos**

Una vez finalizada la validación de los modelos Decision Tree, KNN y Random Forest para el conjunto de datos propuesto, se obtienen los siguientes resultados:

*   **Decision Tree (Modelo Original):** 94.5%.
*   **Decision Tree (Modelo re-entrenado con 'gini'):** 98.5%.
*   **K-Nearest Neighbor:** 94.6%.
*   **Random Forest (Modelo Original):** 69%
*   **Random Forest (Modelo re-entrenado):** 80%

De acuerdo con lo anterior, si bien se observa que el modelo Decision Tree entrenado a través de la función 'gini' obtuvo el mayor porcentaje de exactitud, este puede estar sufriendo de sobreajuste dado que esta es una de las debilidades asociadas al modelo indicado, por tal motivo se concluye que el modelo más optimo según los resultados obtenidos corresponde al Random Forest (re-entrenado) dado que este toma las variables características más influyentes al momento de generar la predicción sobre la variable objetivo por lo que los resultados del modelo serán más acertados.

